In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class down(nn.Module):
    def __init__(self, din, dout):
        super().__init__()
        self.down = nn.Sequential(
            nn.Conv2d(din, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
            nn.Conv2d(dout, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.down(x)
    
class up(nn.Module):
    def __init__(self, din, dout):
        super().__init__()
        self.up = nn.Sequential(
            nn.Conv2d(din, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
            nn.Conv2d(dout, dout, kernel_size=3, padding=1),
            nn.BatchNorm2d(dout),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.up(x)

class unet(nn.Module):
    def __init__(self):
        super().__init__()
        self.scale = 3 # maybe using the original channels might be better
        self.flatten = nn.Flatten()
        self.down1 = down(3, self.scale * 64)
        self.down2 = down(self.scale * 64, self.scale * 128)
        self.down3 = down(self.scale * 128, self.scale * 256)
        self.down4 = down(self.scale * 256, self.scale * 512)
        self.down5 = down(self.scale * 512, self.scale * 1024)
        self.up1 = up(self.scale * 1024, self.scale * 512)
        self.up2 = up(self.scale * 512, self.scale * 256)
        self.up3 = up(self.scale * 256, self.scale * 128)
        self.up4 = up(self.scale * 128, self.scale * 64)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2)
        self.upconv1 = nn.Conv2d(self.scale * 1024, self.scale * 512, kernel_size=3, padding=1)
        self.upconv2 = nn.Conv2d(self.scale * 512, self.scale * 256, kernel_size=3, padding=1)
        self.upconv3 = nn.Conv2d(self.scale * 256, self.scale * 128, kernel_size=3, padding=1)
        self.upconv4 = nn.Conv2d(self.scale * 128, self.scale * 64, kernel_size=3, padding=1)
        self.output = nn.Sequential(
            nn.Conv2d(self.scale * 64, 4, kernel_size=1),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x1 = self.down1(x)
        x2 = self.down2(self.maxpool(x1))
        x3 = self.down3(self.maxpool(x2))
        x4 = self.down4(self.maxpool(x3))
        x5 = self.down5(self.maxpool(x4))
        x6 = self.up1(torch.cat([x4, self.upconv1(self.upsample(x5))], dim=1))
        x7 = self.up2(torch.cat([x3, self.upconv2(self.upsample(x6))], dim=1))
        x8 = self.up3(torch.cat([x2, self.upconv3(self.upsample(x7))], dim=1))
        x9 = self.up4(torch.cat([x1, self.upconv4(self.upsample(x8))], dim=1))
        pre_output = self.output(x9)
        return torch.argmax(pre_output, dim=1)



In [24]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, PILToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


import os
from torchvision.io import decode_image

class dataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_names = sorted([os.path.splitext(filename)[0] for filename in os.listdir(img_dir)])
        self.len = len(self.img_names)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        img = decode_image(os.path.join(self.img_dir, self.img_names[idx] + ".jpg")).float()/255
        label = decode_image(os.path.join(self.label_dir, self.img_names[idx] + ".png"))

        if self.transform:
            img = self.transform(img)

        if self.target_transform:
            label = self.target_transform(label)

        return img.unsqueeze(0), label
    
def display_img_label(data, idx):
    img, label = data[idx]
    figure = plt.figure(figsize=(10,20))
    figure.add_subplot(1, 2, 1)
    plt.imshow(img.permute(1, 2, 0))

    figure.add_subplot(1, 2, 2)
    plt.imshow(label.permute(1, 2, 0), cmap='grey')

    plt.show()

class target_remap(object):
    def __call__(self, img):
        img[img == 255] = 3
        return img


training_data = dataset("rtrain/color", "rtrain/label", target_transform=target_remap())

test_data = dataset("rtest/color", "rtest/label", target_transform=target_remap())

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * 64 + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()

    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \nAvg loss: {test_loss:>8f} \n")

model = unet()

img, label = training_data[0]

res = model(img)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

epochs = 10
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader, model, loss_fn, optimizer)
#     test_loop(test_dataloader, model, loss_fn)
# print("Done!")


tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)
